<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

In [2]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [3]:
#ticker_yahoo = yf.Ticker('HINDZINC.NS')
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)


In [4]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [5]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')
  


def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    #print(f"{ticker} {info['last_price']} {info['year_high']}")
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ath_list))
    return results

results = paralel_func(ath_list)


In [6]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)

#ticker_list.to_csv('Watchlist.csv', index=False)
#ticker_list


In [7]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today()

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df = yf.download(tickers, start= startdate, end= enddate)


[*********************100%***********************]  113 of 113 completed


In [11]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

df1 = pd.DataFrame()


for ticker in tickers:
  test = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
  test1 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
  test2 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
  test3 = ta.ema(close=ax_df[('High', ticker)], length=20).to_frame('EMA200_H')
  test4 = ta.ema(close=ax_df[('Close', ticker)], length=20).to_frame('EMA200_C')
  test5 = ta.ema(close=ax_df[('Low', ticker)], length=20).to_frame('EMA200_L')
  test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
  test1.columns = pd.MultiIndex.from_product([test1.columns, [ticker]])
  test2.columns = pd.MultiIndex.from_product([test2.columns, [ticker]])
  test3.columns = pd.MultiIndex.from_product([test3.columns, [ticker]])
  test4.columns = pd.MultiIndex.from_product([test4.columns, [ticker]])
  test5.columns = pd.MultiIndex.from_product([test5.columns, [ticker]])
  df1 = pd.concat([df1, test, test1, test2, test3, test4, test5], axis=1)
  df1.sort_index()
  
ax_df = pd.concat([ax_df, df1], axis=1)
ax_df.reset_index()
ax_df = ax_df.drop(['Adj Close','Open','Low','High','Volume'], level=0, axis=1)


In [24]:
ax_df= ax_df.swaplevel(0, 1, 1).sort_index(axis=1)

In [54]:
ax_df=ax_df.tail(1)

In [55]:
ax_df

360ONE.NS                                         \
                                 Close     EMA200_C     EMA200_H     EMA200_L   
Date                                                                            
2023-02-06 00:00:00+05:30  1861.699951  1861.672868  1892.721733  1825.583082   

                                                                     \
                          SUPERT_10_1.0 SUPERT_11_2.0 SUPERT_12_3.0   
Date                                                                  
2023-02-06 00:00:00+05:30   1828.761022   1964.627137   1790.942096   

                                                                        ...  \
                          SUPERTd_10_1.0 SUPERTd_11_2.0 SUPERTd_12_3.0  ...   
Date                                                                    ...   
2023-02-06 00:00:00+05:30              1             -1              1  ...   

                           ZFCVINDIA.NS                                \
                          SUPERT_12_3.0 SUPERTd_10_1.0 SUPERTd_11_2.0   
Date                                                                    
2023-02-06 00:00:00+05:30   8996.625048              1              1   

                                                                        \
                          SUPERTd_12_3.0 SUPERTl_10_1.0 SUPERTl_11_2.0   
Date                                                                     
2023-02-06 00:00:00+05:30              1    9403.805484     9199.79514   

                                                                        \
                          SUPERTl_12_3.0 SUPERTs_10_1.0 SUPERTs_11_2.0   
Date                                                                     
2023-02-06 00:00:00+05:30    8996.625048            NaN            NaN   

                                          
                          SUPERTs_12_3.0  
Date                                      
2023-02-06 00:00:00+05:30            NaN  

[1 rows x 1808 columns]

In [104]:
# ax_df1 = pd.DataFrame()

# for ticker in tickers:
#     condition = ax_df[(ticker,'Close')] > ax_df[(ticker,'EMA200_H')]
#     ax_df[condition]

#ax_df[ax_df.xs[('360ONE.NS','Close')] > ax_df.xs[('360ONE.NS','EMA200_H')]]

ax_df.iloc[(0,2)]

1892.72173329255

In [74]:
ax_df1.to_csv('ax_df1.csv')

In [71]:
# ax_df.loc[:,('360ONE.NS','EMA200_H')]
ax_df[('360ONE.NS','EMA200_H')]

Date
2023-02-06 00:00:00+05:30    1892.721733
Name: (360ONE.NS, EMA200_H), dtype: float64